# Выбор локации для скважины

Нужно определить, в каком регионе выгоднее осуществлять добычу нефти.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили три показателя скважины и объём её запасов.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

План работы:
- Загрузить и изучить данные;
- Обучить модели для каждого региона;
- Подготовить функцию и констатные в рамках задачи значения к расчету прибыли;
- Расчитать прибыль и риски для каждого региона, с использованием техники *Bootstrap*;
- Сделать выводы и определить рекомендуемый регион.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data0.info()
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 100000 entries, 0 to 99999

Data columns (total 5 columns):

 #   Column   Non-Null Count   Dtype  

---  ------   --------------   -----  

 0   id       100000 non-null  object 

 1   f0       100000 non-null  float64

 2   f1       100000 non-null  float64

 3   f2       100000 non-null  float64

 4   product  100000 non-null  float64

dtypes: float64(4), object(1)

memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 100000 entries, 0 to 99999

Data columns (total 5 columns):

 #   Column   Non-Null Count   Dtype  

---  ------   --------------   -----  

 0   id       100000 non-null  object 

 1   f0       100000 non-null  float64

 2   f1       100000 non-null  float64

 3   f2       100000 non-null  float64

 4   product  100000 non-null  float64

dtypes: float64(4), object(1)

memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 100000 entries, 0 to 99999

Data columns (total 5 columns

**Вывод:** Имеются данные по трем регионам по 100000 скважин в каждом. Пропуски в данных отсутсвуют. Признаками для обучения модели являются *f0, f1, f2*, а целью значение *product*.

## Обучение и проверка модели

In [4]:
#разделим данные на признаки и результаты
target0 = data0['product']
features0 = data0.drop(['id', 'product'], axis=1)

target1 = data1['product']
features1 = data1.drop(['id', 'product'], axis=1)

target2 = data2['product']
features2 = data2.drop(['id', 'product'], axis=1)

In [5]:
#Разобьём данные на обучающую и валидационную выборки в соотношении 75:25
features0_train, features0_valid, target0_train, target0_valid = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

features1_train, features1_valid, target1_train, target1_valid = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

features2_train, features2_valid, target2_train, target2_valid = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

In [6]:
#Обучим модель и сделаем предсказания на валидационной выборке
#Используем модель линейной регрессии, так как другие модели недостаточно предсказуемые
model0 = LinearRegression()
model0.fit(features0_train, target0_train)
predicted0_valid = model0.predict(features0_valid)

model1 = LinearRegression()
model1.fit(features1_train, target1_train)
predicted1_valid = model1.predict(features1_valid)

model2 = LinearRegression()
model2.fit(features2_train, target2_train)
predicted2_valid = model2.predict(features2_valid)

In [7]:
print('Показатели первого региона')
print('Средний запас предсказанного сырья =', predicted0_valid.mean(), 'RMSE =', 
      mean_squared_error(target0_valid, predicted0_valid)**0.5)

Показатели первого региона

Средний запас предсказанного сырья = 92.59256778438035 RMSE = 37.5794217150813


In [8]:
print('Показатели второго региона')
print('Средний запас предсказанного сырья =', predicted1_valid.mean(), 'RMSE =', 
      mean_squared_error(target1_valid, predicted1_valid)**0.5)

Показатели второго региона

Средний запас предсказанного сырья = 68.728546895446 RMSE = 0.893099286775617


In [9]:
print('Показатели третьегго региона')
print('Средний запас предсказанного сырья =', predicted2_valid.mean(), 'RMSE =', 
      mean_squared_error(target2_valid, predicted2_valid)**0.5)

Показатели третьегго региона

Средний запас предсказанного сырья = 94.96504596800489 RMSE = 40.02970873393434


**Вывод:** Предсказанное количество сырья в первом и третьем регионе близкое по значению, а так же большое значение ошибки до 40% от среднего запаса. Во втором регионе средний запас предсказанного сырья значительно ниже, чем в двух других, но так же в нем значительно меньшее значение показателя RMSE.

## Подготовка к расчёту прибыли

In [10]:
#Сохраним ключевые значения для расчета прибыли
#Сумма, необходимая, чтобы начать разработку 200 скважин и разведку 500
budget = 10000000000
#Доход получаемый с единицы продукта
income_per_unit = 450000
#Количество разрабатываемых скважин
quantity = 200

In [11]:
#Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
min_volume = budget / income_per_unit / quantity
print(min_volume)

111.11111111111111


In [12]:
for p in [target0_valid, target1_valid, target2_valid]:
    print(p.mean())

92.07859674082927

68.72313602435997

94.88423280885438


**Вывод** при затратах на разработку в 10млрд и доходе с каждой единицы продукта 450тыс, при разработке 200 скважин, в каждой из них должно быть не меньше 112 единиц продукта, чтобы покрыть расходы.

Среднее значение объема скважин во всех регионах ниже этого значения.

In [13]:
def income (target, prediction, quantity):
    max_predict = prediction.sort_values(ascending=False)
    target = target.reset_index(drop=True)
    max_wells = target[max_predict.index][:quantity]
    sum_target = max_wells.sum()
    return (sum_target * income_per_unit)

## Расчёт прибыли и рисков 

In [14]:
def profit_risc (targets, predictions):
    values = []
    state = np.random.RandomState(12345)
    for i in range(1000):        
        target = targets.reset_index(drop=True)
        target = target.sample(500, replace=True, random_state=state)        
        prediction = pd.Series(predictions[target.index])
        incomes = income(target, prediction, quantity)
        values.append(incomes-budget)
    
    k = 0
    for i in values:
        if i<0:
            k+=1
            
    values = pd.DataFrame(values)
    quant_0025 = values.quantile(0.025)
    quant_0975 = values.quantile(0.975)
    risc = k / len(values)
    mean = values.mean()   

    print('Среднее значение прибыли =', mean[0])
    print('Нижние 2.5% =', quant_0025[0], ', верхние 2.5% =', quant_0975[0])
    print('Соотношние отрицательной прибыли к количеству проверок =', risc)
    
    return quant_0975, mean, risc

In [15]:
for t, p, r in [
    (target0_valid, predicted0_valid, 'первого'),
    (target1_valid, predicted1_valid, 'второго'),
    (target2_valid, predicted2_valid, 'третьего')
]:
    print(f'Для {r} региона показатели полученные при 1000 выборок по 500 скважин следующие')
    profit_risc(t, p)
    print()

Для первого региона показатели полученные при 1000 выборок по 500 скважин следующие

Среднее значение прибыли = 396164984.8023711

Нижние 2.5% = -111215545.89049526 , верхние 2.5% = 909766941.5534226

Соотношние отрицательной прибыли к количеству проверок = 0.069



Для второго региона показатели полученные при 1000 выборок по 500 скважин следующие

Среднее значение прибыли = 456045105.7866608

Нижние 2.5% = 33820509.39898363 , верхние 2.5% = 852289453.866036

Соотношние отрицательной прибыли к количеству проверок = 0.015



Для третьего региона показатели полученные при 1000 выборок по 500 скважин следующие

Среднее значение прибыли = 404403866.5683568

Нижние 2.5% = -163350413.39560106 , верхние 2.5% = 950359574.9237995

Соотношние отрицательной прибыли к количеству проверок = 0.076




**Вывод:** после получения распределения возможной прибыли при использовании техники Bootstrap, обнаружены следующие результаты:
- Для первого и третьего региона вероятность разрабатывать убыточные скважины более 0,07
- Для второго региона эта вероятно составляет 0,015

Таким образом, наиболее подходящим регионо для разработки является второй, в нем наименьшая веротяность убытков и высокий поазатель средней прибыли.